In [3]:
import flag_analysis

In [4]:
flag_analysis.main()

Step 1: Proving the circuit is etracts syndrome correctly when no fault in the circuit
Stabilizer 0 formula: Xor(Xor(Xor(Xor(False, q3_z), q4_z), q5_z), q6_z)
Stabilizer 1 formula: Xor(Xor(Xor(Xor(False, q1_z), q2_z), q5_z), q6_z)
Stabilizer 2 formula: Xor(Xor(Xor(Xor(False, q0_z), q2_z), q4_z), q6_z)
Stabilizer 3 formula: Xor(Xor(Xor(Xor(False, q3_x), q4_x), q5_x), q6_x)
Stabilizer 4 formula: Xor(Xor(Xor(Xor(False, q1_x), q2_x), q5_x), q6_x)
Stabilizer 5 formula: Xor(Xor(Xor(Xor(False, q0_x), q2_x), q4_x), q6_x)
Result of ordered ancilla vs stabilizer check:
Success : ancilla measurements match stabilizers in order.

Step 2: Finding bad locations in the circuit
Gate index 0: Safe
Gate index 1: Safe
Gate index 2: Bad location 
Gate index 3: Bad location 
Gate index 4: Safe
Gate index 5: Safe
Gate index 6: Safe
Gate index 7: Safe
Gate index 8: Safe
Gate index 9: Bad location 
Gate index 10: Bad location 
Gate index 11: Safe
Gate index 12: Safe
Gate index 13: Safe
Gate index 14: Safe
Gat

In [2]:
# ---------- helper: evaluate z3 Bool expr(s) under a partial assignment ----------
from z3 import Bool, BoolVal, simplify, substitute, is_true

def eval_with_values(exprs, assignment, default_false=True):
    """
    Evaluate Z3 Bool expr(s) under dict name->bool.
    Missing vars default to False iff default_false=True.
    """
    def _eval_one(e):
        subs = []
        # substitute given vars
        for k, v in assignment.items():
            subs.append((Bool(k), BoolVal(v)))
        # optionally ground all *other* variables in e to False
        if default_false:
            # A conservative sweep: try substituting any unknown symbol by name.
            # This is crude but works well for pure-Bool formulas.
            seen = set(assignment.keys())
            stack = [e]
            names_to_false = set()
            while stack:
                node = stack.pop()
                for ch in node.children():
                    stack.append(ch)
                # z3 prints var names as strings when str()'d
                s = str(node)
                if s.isidentifier() and s not in seen and s not in names_to_false:
                    names_to_false.add(s)
            for nm in names_to_false:
                subs.append((Bool(nm), BoolVal(False)))
        val = simplify(substitute(e, subs))
        return bool(is_true(val))
    if isinstance(exprs, (list, tuple)):
        return [_eval_one(e) for e in exprs]
    return _eval_one(exprs)

# ---------- sweep all g0..g3 for a fixed fault ----------
from itertools import product

def sweep_g_cases_for_fault(b_list, fault_name="faulty_gate16_x1", F_list=None, default_false=True):
    """
    b_list : list[BoolRef]  (your data-qubit error indicators)
    fault_name : the single fault variable to set True (others default False)
    F_list : optional list[BoolRef] of flag flip bits; if given, prints flags=Or(F_list)
    """
    rows = []
    g_names = ["g0", "g1", "g2", "g3"]
    for bits in product([False, True], repeat=len(g_names)):
        asgmt = {fault_name: True}
        asgmt.update({n: v for n, v in zip(g_names, bits)})
        b_vals = eval_with_values(b_list, asgmt, default_false=default_false)
        weight = sum(b_vals)
        le1 = (weight <= 1)
        if F_list is not None:
            flags_val = any(eval_with_values(F_list, asgmt, default_false=default_false))
        else:
            flags_val = None
        rows.append((tuple(bits), b_vals, weight, le1, flags_val))
    return rows

def pretty_print_rows(rows, show_flags=True):
    for g_bits, b_vals, wt, le1, flags in rows:
        line = f"g={g_bits} -> b={b_vals}, weight={wt}, sum(b)<=1? {le1}"
        if show_flags and flags is not None:
            line += f", flags={flags}"
        print(line)

# ---------- HOW TO USE ----------
# 1) Ensure `b` is your list of 5 BoolRefs (as you showed), built in your session.
# 2) Optionally define F (e.g., F=[F0, F1]) if you want to report flags.

# Example call (adjust names to your session):
# rows = sweep_g_cases_for_fault(b, fault_name="faulty_gate16_x1", F_list=F, default_false=True)
# pretty_print_rows(rows)

In [4]:
rows = sweep_g_cases_for_fault(b, fault_name="faulty_gate8_x1", default_false=True)
pretty_print_rows(rows)

NameError: name 'b' is not defined